In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch 

output_dir = "fine_tuned_model"

model = AutoModelForCausalLM.from_pretrained(output_dir)
tok = AutoTokenizer.from_pretrained(output_dir)

stop_token_ids = tok("<|im_end|>")["input_ids"]

prompt = "How many 'r' are in raspberry"
prompt = "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n" + prompt + "<|im_end|>\n<|im_start|>assistant\n"

inputs = tok(prompt, return_tensors="pt")

max_new_tokens = 500
generated_ids = inputs["input_ids"].clone()

for _ in range(max_new_tokens):
    outputs = model(input_ids=generated_ids)
    next_token_id = torch.argmax(outputs.logits[:, -1, :], dim=-1).unsqueeze(0)
    generated_ids = torch.cat([generated_ids, next_token_id], dim=1)
    token_text = tok.decode(next_token_id[0], skip_special_tokens=True)
    print(token_text, end="", flush=True)

print("\nDone.")